In [1]:
  import rioxarray as rioxa
from glob import glob
from dask.distributed import Client, LocalCluster, Lock
import os
import s3fs

# s3 = s3fs.S3FileSystem()

# def timeseries_creator_op(raster_root, key='etf_*', suffix='.tif', recur=False):
#     """Takes a
#      directory full of etf timeseries and makes a dictionary of
#      {key(raster_name): (subject_rasterpath, [before_lst], [after_lst])}, where before_lst and after_lst are paths of
#      rasters that come temporally before and after the 'subject_rasterpath'."""

#     # print(f'{raster_root}{os.sep}{key}{suffix}')
#     # print(glob(f'{raster_root}{os.sep}{key}{suffix}'))
#     rasters_list = sorted(s3.glob(f'{raster_root}{os.sep}{key}{suffix}'))
#     print('raster list')
#     print(rasters_list)

#     ras_dict = {}
#     for i, ras in enumerate(rasters_list):
#         # get the rastername
#         # rasname = s3fs.split_path(ras)[1]
#         rasname = ras.split('/')[-1]
#         print('raslist to split')
#         print(ras.split('/'))
#         # grab before and after rasters where possible
#         if i > 0:
#             before_rasters = rasters_list[0:i]
#             # we reverse order the before rasters... for going backwards...
#             before_rasters.reverse()
#         if i < (len(rasters_list) - 1):
#             after_rasters = rasters_list[i+1:]
#         # if at the beginning or end, put in none for the last value.
#         if i == 0:
#             value_tuple = (ras, None, after_rasters)
#         elif i == (len(rasters_list) - 1):
#             value_tuple = (ras, before_rasters, None)
#         else:
#             value_tuple = (ras, before_rasters, after_rasters)


#         ras_dict[rasname] = value_tuple

#     return ras_dict


# def baba_gapfill_op(output_location, name, path, before_lst, index=2, client=None, climo_fill=None):
#     """"""
#     # todo - check to make sure that index < before_list and index < after_lst

#     print('name: ', name)
#     print(before_lst)


#     raster_ds = rioxa.open_rasterio(path, chunks='auto').squeeze().drop(labels='band')

#     for i in range(index):
#         # open the fill raster
#         try:
#             fill_ras = before_lst[i]
#             a_ds = rioxa.open_rasterio(fill_ras, chunks='auto').squeeze().drop(labels='band')
#             #  fill raster_ds with the fill value
#             raster_ds = raster_ds.fillna(value=a_ds)
#         except IndexError:
#             # if there isn't enough fill data
#             raster_ds = raster_ds.fillna(value=0.75)
#         except TypeError:
#             # if the raster is the start of the series
#             raster_ds = raster_ds.fillna(value=0.75)

#     if climo_fill is not None:
#         climo_ds = rioxa.open_rasterio(climo_fill, chunks='auto').squeeze().drop(labels='band')
#         raster_ds = raster_ds.fillna(value=climo_ds)
#     # even if there are two fill rasters available in the before list. still fill gaps with 0.75
#     # this doesn't do anything if the raster was already gapfilled by 0.75 during the exceptions above
#     raster_ds = raster_ds.fillna(value=0.75)

#     raster_ds.rio.to_raster(os.path.join(output_location, name),
#                             tiled=True, lock=Lock('rio', client=client))

In [2]:
def baba_gapfill_op_simple(output_location, name, current_etf, prev_raster_list, client=None, climo_fill=None):
    """"""
    # todo - check to make sure that index < before_list and index < after_lst

    # print('name: ', name)
    # print(before_lst)

    path = current_etf
    raster_ds = rioxa.open_rasterio(path, chunks='auto').squeeze().drop(labels='band')

    before_lst = prev_raster_list
    for i in range(2):
        # open the fill raster
        try:
            fill_ras = before_lst[i]
            a_ds = rioxa.open_rasterio(fill_ras, chunks='auto').squeeze().drop(labels='band')
            #  fill raster_ds with the fill value
            raster_ds = raster_ds.fillna(value=a_ds)
        except IndexError:
            # if there isn't enough fill data
            raster_ds = raster_ds.fillna(value=0.75)
        except TypeError:
            # if the raster is the start of the series
            raster_ds = raster_ds.fillna(value=0.75)

    if climo_fill is not None:
        climo_ds = rioxa.open_rasterio(climo_fill, chunks='auto').squeeze().drop(labels='band')
        raster_ds = raster_ds.fillna(value=climo_ds)
    # even if there are two fill rasters available in the before list. still fill gaps with 0.75
    # this doesn't do anything if the raster was already gapfilled by 0.75 during the exceptions above
    raster_ds = raster_ds.fillna(value=0.75)

    raster_ds.rio.to_raster(os.path.join(output_location, name),
                            compress='LZW', tiled=True, lock=Lock('rio', client=client))

In [3]:
    # Initialize local hardware with Dask
    cluster = LocalCluster()
    client = Client(cluster)
    print(client)
    print(cluster)

<Client: 'tcp://127.0.0.1:42095' processes=4 threads=16, memory=61.58 GiB>
LocalCluster(b56db65e, 'tcp://127.0.0.1:42095', workers=4, threads=16, memory=61.58 GiB)


In [4]:
    # where the rasters are...
    raster_root = r's3://ws-out/ssebop_viirs/'
    # set up an output location
    output_path = '/wsefs/pipeline/babadata/'
    #ras_dict = timeseries_creator_op(raster_root=raster_root)

In [5]:
prev_raster_list = ['/wsefs/pipeline/etfdata/etf_2022101.tif',
                    '/wsefs/pipeline/etfdata/etf_2022093.tif']
current_etf = '/wsefs/pipeline/etfdata/etf_2022102.tif'
name='etfca_2022102.tif'

baba_gapfill_op_simple(output_path, name, current_etf, prev_raster_list, client=client, climo_fill=None)

In [6]:
! ls -lh /wsefs/pipeline/babadata/

total 490M
-rw-rw-r-- 1 ec2-user ec2-user 490M Dec 22 20:35 etfca_2022102.tif


In [7]:
! ls /wsefs/pipeline/sdata/

LS_eVSE_TEMP.2022.284-293.1KM.LST_ACQT.006.2022294042535.txt
LS_eVSE_TEMP.2022.284-293.1KM.LST_ANGL.006.2022294042535.tif
LS_eVSE_TEMP.2022.284-293.1KM.LST_EM14.006.2022294042535.tif
LS_eVSE_TEMP.2022.284-293.1KM.LST_EM15.006.2022294042535.tif
LS_eVSE_TEMP.2022.284-293.1KM.LST_NMBR.006.2022294042535.tif
LS_eVSE_TEMP.2022.284-293.1KM.LST_QLTY.006.2022294042535.tif
LS_eVSE_TEMP.2022.284-293.1KM.LST_TEMP.006.2022294042535.tif


In [8]:
! ls /wsefs/pipeline/etfdata/

etf_2022093.tif  etf_2022101.tif  etf_2022102.tif  etf_test.tif


In [9]:
#! cp /wsefs/pipeline/etfdata/etf_test.tif /wsefs/pipeline/etfdata/etf_2022102.tif

In [10]:
! ls -lh /wsefs/pipeline/etfdata/

total 8.2G
-rw-r--r-- 1 ec2-user ec2-user 2.1G Dec 22 20:02 etf_2022093.tif
-rw-r--r-- 1 ec2-user ec2-user 2.1G Dec 22 20:02 etf_2022101.tif
-rw-r--r-- 1 ec2-user ec2-user 2.1G Dec 22 20:03 etf_2022102.tif
-rw-r--r-- 1 ec2-user ec2-user 2.1G Dec 19 20:34 etf_test.tif
